In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from Models.AutoEncoder import AutoEncoder, AutoEncoderDataset
from utils.utils import *
from tqdm import tqdm

# Auto Encoder Test

## Base Setup

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
num_epochs = 50
batch_size = 128
lr = 1e-4

In [3]:
# Feature Selection
cat_features = ['Gender', 'Card Brand', 'Card Type', 'Expires', 'Has Chip', 'Year PIN last Changed', 'Whether Security Chip is Used', 'Day', 'Error Message']

num_features = ['Current Age', 'Retirement Age', 'Per Capita Income - Zipcode', 'Yearly Income', 'Total Debt', 'Credit Score', 'Credit Limit', 'Amount', 'Since Open Month']

discarded = ['User', 'Birth Year', 'Birth Month', 'Card', 'Card Number', 'Zipcode', 'Merchandise Code', 'Acct Open Date']


In [5]:
tmp = pd.read_csv('Data/[24-2 DS_Project2] Data.csv')
print(tmp.columns)

NameError: name 'pd' is not defined

In [1]:
# 데이터 전처리
data_path = 'Data/[24-2 DS_Project2] Data.csv'
(train_cat_X, train_num_X, train_y), (valid_cat_X, valid_num_X, valid_y), label_encoders = process_data(
    data_path,
    cat_features,
    num_features,
    discarded
)

NameError: name 'process_data' is not defined

In [ ]:
train_dataset = AutoEncoderDataset(train_cat_X, train_num_X, device)
valid_dataset = AutoEncoderDataset(valid_cat_X, valid_num_X, device)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)

In [ ]:
model = AutoEncoder(encoding_dim=16, cat_features=cat_features, num_features=num_features).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [ ]:
for epoch in range(100):
    model.train()
    train_loss = 0
    for batch_idx, (cat_features, num_features) in tqdm(enumerate(train_loader), total=len(train_loader), desc=f'에포크 {epoch+1}', bar_format='{l_bar}{bar:20}{r_bar}'):
        optimizer.zero_grad()
        y_hat, y = model(cat_features, num_features)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f'에포크 {epoch+1}/100 | 평균 학습 손실: {avg_train_loss:.6f}')